# Increasing the Notebook Size

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Importing Native Python Libraries
import os
import math
import random
import argparse
import collections
import re
from collections import namedtuple, deque, defaultdict
from itertools import count

# Importing Numpy and Pandas
import numpy as np
import pandas as pd
# import modin.pandas as pd

# Importing Matplotlib
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

# Importing TA-Lib
import talib

# import PyTorch LIbraries
import torch
import torch.utils.data as utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

# Importing SciKit Learn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale

# Importing RL Libraries
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, Box

# Importing TQDM to track cell progress
from tqdm import tqdm

# Ignoring Warnings
# import warnings

# warnings.filterwarnings('ignore')

In [3]:
# Importing Custom Code
from MainCode.env import Simulator, Window, CustomTensorDataReader

make sure to add .gitignore to temp_directory
file already exist


# Initializing Custom Classes Built from Pre-processing Files

In [4]:
# Initializing our class to read our numpy matrix files and merging them
dr = CustomTensorDataReader()

In [5]:
# Initializing our class to manage our environment Window
# This is our login window (1 second time stamp)
wd = Window()

# Showing the shape of our data with 

In [6]:
# 62223 Time Stamps
# 1553 Logins for each time stamp (this includes padding entries with 0)
# 17 items per login entry
dr.data.shape

(62223, 1553, 17)

In [7]:
observation, info = wd.reset_window()

In [10]:
observation.shape

(1553, 17)

In [11]:
wd.target_window

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

In [13]:
wd.current_index

29813

In [37]:
# This is how to access our data
# This accesses the first login, represented by j, of our first time stamp, i
# wd.data[i][j]
wd.data.shape

(62223, 1553, 17)

In [35]:
wd.target_data.shape

(14863575,)

In [51]:
if (1 not in wd.target_data):
    print('Not in')
else:
    print("In")

# if (np.any(wd.target_data, 1) == True):
#     print('Got it')
# else:
#     print('damn')

In


In [112]:
class NetworkEnv(Env):
    def __init__(self):
        # Actions we can take: We can Flag an item, Continue to the next item, or Finish/Hold once we have analyzed the current log/activity
        self.action_space = Discrete(3)
        
        # Time Stamp Login Array
        self.observation_space = Box(low= np.array([0]), high = np.array([1552]), dtype=int)
        
        # Set Starting Login Entry
        self.state = random.randint(0,1552)
        
        # Set Network Log Length
        self.log_length = 1553
        
#         # Set Network Health
        self.network_health = 100

        
    def step(self, action):
        # Apply Actions
        # -1 If we have run out of items in the list we want to declare that we are finished OR if we are in a live environment we want to hold until our next activity
        # 0 If we flag the item we want to perform an action before we continue OR if we are holding we want to flag when we start
        # 1 If we don't detect abnormal behavior we want to continue to process the next item in the list
        self.state += action
        
        # Decreamenting the length of our log by 1
        self.log_length -= 1 
        
        # Calculating Reward
        # Cases which our agent is correct
        if (self.state == 1) and (1 in wd.target_window):
            reward = 25 
        elif (self.state == 0) and (1 not in wd.target_window):
            reward = 25   
        # Cases which our agent is incorrect    
        elif (self.state == 1) and (1 not in wd.target_window):
            reward = 50  
        elif (self.state == 0) and (1 in wd. target_window):
            reward = 50
            self.network_health -= 25
            
        if self.log_length == 0 or self.network_health <= 0:
            done = True
        else:
            done = False
        
        # Placeholder for info
        info = {}
    
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        # Reset our environment
        self.state = random.randint(0,1552)
        # Reset our log length
        self.log_length = 1553
        # Reset our network health
        self.network_health = 100
        
        return self.state


In [113]:
env = NetworkEnv()

In [114]:
env.observation_space.sample()

array([830])

In [115]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
#     network_health = 100
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
#         network_health += health
    print('Episode:{} Score:{}'.format(episode, score))

UnboundLocalError: local variable 'reward' referenced before assignment